# Claim 2.1: HS improves predictive performance of RF

In [1]:
# imports
%load_ext autoreload
%autoreload 2

# basic data science utilities
import numpy as np
import pandas as pd

# basic system utilities 
import os
import sys

# datasets used by authors in paper
import imodels
from imodels.util.data_util import get_clean_dataset # this was used to get the datasets

# copying Random Forests (RF) so that I can use HS on them (not used because imodels already implements a way to run HS on RFs)
# from copy import deepcopy

# sklearn baseline random forest 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# cross-validation of models & model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, auc, make_scorer

from sklearn.model_selection import cross_val_score

# hyperparameter tunnning
from skopt import gp_minimize

# hierarchical shrinkage
from imodels import HSTreeClassifier, HSTreeClassifierCV 

# bayesian-additive regression models (BART)
from bartpy.sklearnmodel import SklearnModel

# timing algorithm execution
import time

In [2]:
# Datasets used in paper (location in author repo: github.com/Yu-Group/imodels-experiments/config/shrinkage/models.py)

DATASETS_CLASSIFICATION = [
    # classification datasets from original random forests paper
    # page 9: https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf
    # ("heart", "heart", 'imodels'),
    # ("breast-cancer", "breast_cancer", 'imodels'),
    # ("haberman", "haberman", 'imodels'),
    # ("ionosphere", "ionosphere", 'pmlb'),
    # ("diabetes", "diabetes", "pmlb"),
    # ("german-credit", "german", "pmlb"),
    ("juvenile", "juvenile_clean", 'imodels'),
    ("recidivism", "compas_two_year_clean", 'imodels')
]

# load datasets (datasets of authors seem to already be preprocessed so we will use theirs)
tasks = {}

for task in DATASETS_CLASSIFICATION:
    X, y, feature_names = get_clean_dataset(task[1], data_source = task[2])
    df = pd.DataFrame(X, columns=feature_names)
    df["label"] = y
    tasks[task[0]] = df

In [ ]:
# dataframe to save performance of models
classification_results = pd.DataFrame(columns = ["task", "dataset", "boot_iter", "algorithm", "scoring", "n_trees", "regularization", "train_score", "test_score", \
                                                 "train_wall_time", "test_wall_time", "train_cpu_time", "test_cpu_time", "tunning_wall_time", "tunning_cpu_time"])

# number of leafs used in paper
num_of_trees = [10, 25, 50, 75, 100, 300, 500]

# regularization parameter
reg_hs = [0.1, 1.0, 10.0, 25.0, 50.0, 100.0]

# number of times to repeat evaluations with random splits (5 repeats with 3-fold cross-validation = 15 repeats)
NUM_OF_BOOTSTRAP_SAMPS = 5

# for each dataset that was used in paper
for task in DATASETS_CLASSIFICATION:
    # repeat NUM_OF_BOOTSTRAP_SAMPS
    for samp in range(NUM_OF_BOOTSTRAP_SAMPS):
        # use statified splitting (we tried both stratified and un-stratified => no significant differences)
        skf = StratifiedKFold(n_splits=3)
         
        X, y = np.array(tasks[task[0]].drop("label", axis = 1)), np.array(tasks[task[0]]["label"])
        
        # cross-validation loop
        for i, (train_index, test_index) in enumerate(skf.split(tasks[task[0]], tasks[task[0]]["label"])):
            print(f"Dataset: {task[0]}, Sample: {samp}, Fold {i}", end = "\r")

            X_train, y_train = X[train_index, :], y[train_index]
            X_test, y_test = X[test_index, :], y[test_index]

            # for each tree (as deduced from fig. 4D)
            for m in num_of_trees:
                
                ### Random Forest (RF) ###
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                rf = RandomForestClassifier(n_estimators=m, max_features = "sqrt").fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_rf = rf.predict(X_train)
                y_test_pred_rf = rf.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["RF"],
                                                                            "scoring": ["AUC"],
                                                                            "n_trees": [m],
                                                                            "regularization": ["None"],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_rf)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_rf)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [None], 
                                                                            "tunning_cpu_time": [None]})])
                
                ### RF-CV (max_features (mtry)) ###                
                
                # tunning function to use in gp_minimize
                def rf_mtry(mtry):
                    rf_mtry = RandomForestClassifier(n_estimators=m, max_features = mtry[0])
                    roc_spec = make_scorer(roc_auc_score, needs_proba=True)
                    scores = cross_val_score(rf_mtry, X_train, y_train, cv=3, scoring = roc_spec)
                    return -np.mean(scores)
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                mtry_best = gp_minimize(rf_mtry,
                            [(0.1, 1.0)],
                            acq_func="EI",
                            n_calls = 15,
                            n_initial_points = 5,
                            noise = 0.1**2).x[0]
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                rf_mtry = RandomForestClassifier(n_estimators=m, max_features = mtry_best).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_rf_mtry = rf_mtry.predict(X_train)
                y_test_pred_rf_mtry = rf_mtry.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["RF-MTRY"],
                                                                            "scoring": ["AUC"],
                                                                            "n_trees": [m],
                                                                            "regularization": [rf_mtry], # we store best mtry parameter in regularization
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_rf_mtry)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_rf_mtry)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])
                
                ### RF-CV (max_depth (depth)) ###
                def rf_depth(depth):
                    rf_depth = RandomForestClassifier(n_estimators=m, max_depth = int(np.round(depth[0])))
                    roc_spec = make_scorer(roc_auc_score, needs_proba=True)
                    scores = cross_val_score(rf_depth, X_train, y_train, cv=3, scoring = roc_spec)
                    return -np.mean(scores)
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                depth_best = int(np.round(gp_minimize(rf_depth,
                            [(1.0, 30.0)],
                            acq_func="EI",
                            n_calls = 15,
                            n_initial_points = 5,
                            noise = 0.1**2).x[0]))
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                rf_depth = RandomForestClassifier(n_estimators=m, max_depth = depth_best).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_rf_depth = rf_depth.predict(X_train)                
                y_test_pred_rf_depth = rf_depth.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["RF-DEPTH"],
                                                                            "scoring": ["AUC"],
                                                                            "n_trees": [m],
                                                                            "regularization": [rf_depth], # we store best depth parameter in regularization
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_rf_depth)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_rf_depth)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])
                
                ### HS-RF (hierarchical shrinkage) ###
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                roc_spec = make_scorer(roc_auc_score)
                rf = RandomForestClassifier(n_estimators=m, max_features = "sqrt")
                hs_rf_cv = HSTreeClassifierCV(estimator_=rf, reg_param_list = reg_hs, cv = 3, scoring = roc_spec)
                hs_rf_cv.fit(X_train, y_train)
                
                best_hs_reg = hs_rf_cv.reg_param
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                rf = RandomForestClassifier(n_estimators=m, max_features = "sqrt")
                hs_rf = HSTreeClassifier(estimator_= rf, reg_param = best_hs_reg) 
                hs_rf.fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_hs_rf = hs_rf.predict(X_train)
                y_test_pred_hs_rf = hs_rf.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["HS-RF"],
                                                                            "scoring": ["AUC"],
                                                                            "n_trees": [m],
                                                                            "regularization": [best_hs_reg], # HS regression parameter (lambda)
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_hs_rf)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_hs_rf)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])
                
                ### BART (doesn't use tunning - takes to long/performs the best anyway) ###
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()

                bart = SklearnModel(n_trees = m);
                bart.fit(X_train, y_train);

                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()

                bart_train_pred = np.round(bart.predict(X_train));
                bart_test_pred = np.round(bart.predict(X_test));

                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                                "dataset": [task[0]],
                                                                                "boot_iter": [samp],
                                                                                "algorithm": ["BART"],
                                                                                "scoring": ["AUC"],
                                                                                "n_trees": [m],
                                                                                "regularization": ["None"],
                                                                                "train_score": [roc_auc_score(y_train, bart_train_pred)],
                                                                                "test_score": [roc_auc_score(y_test, bart_test_pred)],
                                                                                "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                                "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                                "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                                "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [None], 
                                                                            "tunning_cpu_time": [None]})])

                classification_results.to_csv("claim_2_1_classification.csv")

  0%|          | 0/200 [00:00<?, ?it/s]


In [ ]:
display(classification_results)